In [5]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from datetime import datetime
import numpy as np

# Lista para armazenar informações sobre produtos
products_list = []

# Função para calcular o WMAPE (corrigida)
def calculate_wmape(y_true, y_pred, weights):
    errors = np.abs(y_true - y_pred)
    weighted_errors = errors * weights
    sum_weighted_errors = np.sum(weighted_errors)
    sum_y_true = np.sum(weights * y_true)
    wmape = (sum_weighted_errors / sum_y_true) * 100
    return wmape

# Função para calcular o WMAPE para cada produto
def calculate_wmape_for_products(products):
    results = []
    for product in products:
        try:
            # Converter valores de texto para listas de floats
            real_values = np.array(product['real_values'])
            predicted_values = np.array(product['predicted_values'])

            # Se pesos não forem fornecidos, assumir pesos iguais
            if not product['weights']:
                weights = np.ones(len(real_values))
            else:
                weights = np.array(product['weights'])

            # Calcular WMAPE
            wmape = calculate_wmape(real_values, predicted_values, weights)

            # Adicionar resultados à lista
            results.append({'Product': product['product'], 'WMAPE': wmape})
        except Exception as e:
            results.append({'Product': product['product'], 'WMAPE': f"Erro: {str(e)}"})
    return results

# Inicializando o aplicativo Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

# Layout do aplicativoLUX
app.layout = dbc.Container(
    [
        dbc.Tabs(
            [
                dbc.Tab(
                    label='Capa',
                    tab_id='Tab-1',
                    children=[
                        dbc.Card(
                            dbc.Row(
                                [
                                    dbc.Col(html.Img(src='https://th.bing.com/th/id/OIP.wOjaHYfiGri4pxMPJ8_rBgHaFH?rs=1&pid=ImgDetMain',
                                                    style={'width': '40%'}), width=6),
                                    dbc.Col(html.Img(src='https://www.predatorsoft.com/wp-content/uploads/2020/02/SOP-01-clear.png', 
                                                    style={'width': '50%'}), width=6), 

                                    dbc.Col(
                                        [
                                            html.H1("Calculadora de WMAPE"),
                                            dbc.Badge('Sales & Operation Planning (S&OP)', color='success', className='ms-1',
                                                      style={'font-size': '24px'})
                                        ],
                                        width=12,
                                    ),
                                ]
                            ),
                            style={'padding': '20px'}  # Adicionando espaçamento interno ao card
                        ),
                        
                        dbc.Row(
                            dbc.Col(
                                dbc.Card(
                                    dbc.CardBody(
                                        [
                                            html.H4("Data e Hora Atuais", className="card-title"),
                                            html.P(id="current-time", className="card-text"),
                                            dcc.Interval(
                                                id="interval-component",
                                                interval=1000,  # Atualiza a cada 1000 milissegundos (1 segundo)
                                                n_intervals=0,
                                            ),
                                        ]
                                    ),
                                    style={'padding': '20px'}  # Adicionando espaçamento interno ao card
                                )
                            )
                        )
                    ]
                ),
                
                dbc.Tab(
                    label='Calculadora WMAPE',
                    tab_id='tab-2',
                    children=[
                        dbc.Card(
                            dbc.Row(
                                [
                                    dbc.Col(html.H1("Calculadora de WMAPE para Previsão de Vendas"), width=12), html.Br(),
                                ]
                            )
                        ),
                        
                        dbc.Card(
                            dbc.Row(
                                [
                                    dbc.Col(html.Label("Produto:")),
                                    dbc.Col(dcc.Input(id='product_input', type='text', value='',
                                    style={'marginRight': '10px'})),

                                    dbc.Col(html.Label("Valores Reais:")),
                                    dbc.Col(dcc.Input(id='real_values_input', type='text', value='', 
                                    style={'marginRight': '10px'})),

                                    dbc.Col(html.Label("Valores Previstos:")),
                                    dbc.Col(dcc.Input(id='predicted_values_input', type='text', value='', 
                                    style={'marginRight': '10px'})),

                                    dbc.Col(html.Label("Pesos (opcional):")),
                                    dbc.Col(dcc.Input(id='weights_input', type='text', value='', 
                                    style={'marginRight': '10px'})),

                                    dbc.Col([html.Button('Adicionar Produto', id='add_product_button', n_clicks=0),
                                    html.Button('Reiniciar', id='reset_button', n_clicks=0)],
                                    style={'marginBottom': '20px'}
                                    ),
                   
                                ]
                            )
                        ),
                        
                        dbc.Card(
                            dbc.Row(
                                [
                                    dbc.Col(
                                        dash_table.DataTable(
                                            id='wmape_table',
                                            columns=[
                                                {'name': 'Produto', 'id': 'Product'},
                                                {'name': 'WMAPE', 'id': 'WMAPE'},
                                            ],
                                        ),
                                        style={'padding': '20px'}  # Adicionando espaçamento interno ao card
                                    ),
                                ]
                            )
                        ),
                        
                        dbc.Card(
                            dbc.Row(
                                [
                                    dbc.Col(
                                        dash_table.DataTable(
                                            id='total_wmape_table',
                                            columns=[
                                                {'name': 'Produto', 'id': 'Product'},
                                                {'name': 'WMAPE Total', 'id': 'Total WMAPE'},
                                            ],
                                        ),
                                        style={'padding': '20px'}  # Adicionando espaçamento interno ao card
                                    ),
                                ]
                            )
                        ),
                    ]
                ),
                
            ],
        ),
    ],
    fluid=True
)

# Callback para adicionar produtos
@app.callback(
    Output('wmape_table', 'data'),
    [Input('add_product_button', 'n_clicks'),
     Input('reset_button', 'n_clicks')],
    [State('product_input', 'value'),
     State('real_values_input', 'value'),
     State('predicted_values_input', 'value'),
     State('weights_input', 'value')]
)
def add_product(n_clicks_add, n_clicks_reset, product, real_values, predicted_values, weights):
    ctx = dash.callback_context
    triggered_id = ctx.triggered_id
    try:
        if 'add_product_button' in triggered_id:
            # Converter valores de texto para listas de floats
            real_values = list(map(float, real_values.split(',')))
            predicted_values = list(map(float, predicted_values.split(',')))
            weights = list(map(float, weights.split(','))) if weights else None
            
            products_list.append({
                'product': product,
                'real_values': real_values,
                'predicted_values': predicted_values,
                'weights': weights
            })
            
        elif 'reset_button' in triggered_id:
            # Reiniciar dados
            products_list.clear()

        # Calcular WMAPE para todos os produtos
        results = calculate_wmape_for_products(products_list)

        # Verificar se houve um erro na função
        if isinstance(results, str):
            raise dash.exceptions.PreventUpdate

        return results
    except Exception as e:
        return [{'Product': 'Erro', 'WMAPE': f"{str(e)}"}]

# Callback para atualizar a tabela de resultados totais
@app.callback(
    Output('total_wmape_table', 'data'),
    Input('add_product_button', 'n_clicks'),
    prevent_initial_call=True
)
def update_total_wmape_table(n_clicks_add):
    try:
        # Calcular WMAPE para todos os produtos
        results = calculate_wmape_for_products(products_list)

        # Calcular o WMAPE total
        total_wmape = np.mean([result['WMAPE'] for result in results])

        # Adicionar resultados à tabela de WMAPE total
        total_results = [{'Produto': 'WMAPE Total', 'Total WMAPE': f"{total_wmape:.2f}%"}]

        return total_results
    except Exception as e:
        return [{'Produto': 'Erro', 'WMAPE': f"{str(e)}"}]

# Atualização da data e hora
@app.callback(
    Output("current-time", "children"),
    Input("interval-component", "n_intervals"),
)
def update_time(n):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Data e Hora Atuais: {current_time}"

# Executando o aplicativo Dash
if __name__ == "__main__":
    app.run_server(debug=True, port=8031)
